In [1]:
import numpy as np

import open3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:

# Each npz file has following fields:

#     data: the xyz coordinates of simulated data with noise;
#     scan: the xyz coordinates of simulated data without noise;
#     ins: the instance label
#     cls: the class label

# The primitives are parameterized with following parameters:

#     Plane: a point on the plane and its norm; i.e. x, y, z, nx, ny, nz.
#     Sphere: a point on the center and its radius; i.e. x, y, z, r.
#     Cylinder: a point on the axis, the axis, and the radius; i.e. x, y, z, nx, ny, nz, r.
#     Cone: the apex, the axis, and the open angle in radius; i.e. x, y, z, nx, ny, nz, \theta


In [3]:
bagsfit_files_path = "/home/pranayspeed/Downloads/TRAIN-20s/"
sequence = 0
test_case=0


In [4]:
def get_prim_data(bagsfit_files_path, sequence, test_case):
    full_path_prim = bagsfit_files_path + f"{sequence:03d}" + "/" +f"{test_case:05d}" +".prim"
    prim_data = []
    with open(full_path_prim, "r") as data:        
        for line in data:
            curr_prim = {}
            curr_data = line.split(" ")
            #print(curr_data[0] )
            curr_prim['type'] = curr_data[0] 
            if curr_prim['type'] == "Plane":
                curr_prim['point'] = np.array([float(curr_data[1]), float(curr_data[2]), float(curr_data[3])])
                curr_prim['norm'] = np.array([float(curr_data[4]), float(curr_data[5]), float(curr_data[6])])
            elif curr_prim['type'] == "Sphere":
                curr_prim['point'] = np.array([float(curr_data[1]), float(curr_data[2]), float(curr_data[3])])
                curr_prim['radius'] = float(curr_data[4])
            elif curr_prim['type'] == "Cylinder":
                curr_prim['point'] = np.array([float(curr_data[1]), float(curr_data[2]), float(curr_data[3])])
                curr_prim['axis'] = np.array([float(curr_data[4]), float(curr_data[5]), float(curr_data[6])])
                curr_prim['radius'] = float(curr_data[7])
            elif curr_prim['type'] == "Cone":
                curr_prim['point'] = np.array([float(curr_data[1]), float(curr_data[2]), float(curr_data[3])])
                curr_prim['axis'] = np.array([float(curr_data[4]), float(curr_data[5]), float(curr_data[6])])
                curr_prim['angle'] = float(curr_data[7])
            prim_data.append(curr_prim)
    return prim_data

def get_test_case_data(bagsfit_files_path, sequence, test_case):
    full_path = bagsfit_files_path + f"{sequence:03d}" + "/" +f"{test_case:05d}" +".npz"
    geom_data={}
    with np.load(full_path) as data:
        geom_data['data'] = data['data'].reshape(3,-1).T
        geom_data['ins'] = np.array(data['ins']).reshape(-1,1).flatten()
        geom_data['cls']= np.array(data['cls']).reshape(-1,1).flatten()
        geom_data['scan'] = data['scan'].reshape(3,-1).T

    geom_data['prim'] = get_prim_data(bagsfit_files_path, sequence, test_case)
    return geom_data




In [5]:
def visualize_test_case(scan_data, cls):
    color_map = {
    -1: [0, 0, 0],
    0: [0, 0, 255],
    1: [0, 255, 0],
    2: [255, 0, 0],
    3: [0, 255, 255],
    }
    color_values = np.array([color_map[x] for x in cls])
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(scan_data)
    pcd.colors = open3d.utility.Vector3dVector(color_values)
    open3d.visualization.draw_geometries([pcd])



In [6]:
# sequence =2
# test_case=3
# geom_data = get_test_case_data(bagsfit_files_path, sequence, test_case)
# visualize_test_case(geom_data['data'], geom_data['cls'])

In [7]:
def visualize_by_computing_normals(scan_data):
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(scan_data)
    pcd.estimate_normals(search_param=open3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=500))
    open3d.visualization.draw_geometries([pcd])

In [8]:
#visualize_by_computing_normals(geom_data['data'])

In [9]:
#torch segmentation CNN model using pointclouds

from GeometrySegmentation import PointCloudSegmentation, PointCloudDataset

import torch
import torch.optim as optim
import torch.nn as nn

In [10]:
# Create the training dataset
train_points = ... # A list of point clouds
train_labels = ... # A list of corresponding labels
train_dataset = PointCloudDataset(bagsfit_files_path)

# Create the data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)


In [12]:
number_of_classes = 5
batch_size=32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model
model = PointCloudSegmentation(num_shapes=number_of_classes)
model = model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(100):
    # Iterate over the training data
    for input_points, labels in train_loader:
        input_points = input_points.view(batch_size, -1, 3)
        print(input_points.shape)
        input_points = input_points.to(device)
        print(input_points.shape)
        labels = labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_points)
        loss = criterion(outputs.view(-1, number_of_classes), labels.view(-1))

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Print the average loss for this epoch
    print("Epoch {}: Loss = {}".format(epoch, loss.item()))

# Save the model
torch.save(model.state_dict(), "point_cloud_segmentation.pt")

torch.Size([32, 307200, 3])
torch.Size([32, 307200, 3])


RuntimeError: Given groups=1, weight of size [64, 3, 1], expected input[32, 307200, 3] to have 3 channels, but got 307200 channels instead